In [1]:
import torch
import json
import requests
from PIL import Image
from io import BytesIO
import numpy as np
from ultralytics import YOLO
import matplotlib.pyplot as plt
import os
import pandas as pd
from tqdm import tqdm 

<h3><i>Download images</i></h3>

In [2]:
df = pd.read_csv('pizza_data_final.csv', index_col='id')
df.head()

C:\Users\DuyAnh\AppData\Local\Temp\ipykernel_17288\2855637729.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('pizza_data_final.csv', index_col='id')


,image_url,store,oven,exception,created_at,updated_at,__v,error_list,model_version,Bánh không tròn | Distorted shape,...,Không cân Topping - Not even half and half,Sốt trên mép | Topping - Sauce covering on the edge,Lên men thiếu | Fermentation - Lack fermentation,Quá theo viền | Topping - Too strong shaping the edge,Trộn lẫn | Topping - Bended,Quá tập trung | Topping - Topping too centered,Topping - Không đều | Not even,Lên men quá nhiều | Fermentation - Over fermentation,Không hình tròn | Topping - Not circled,Bánh lớn | Size - Too big
id,,,,,,,,,,,,,,,,,,,,,
644a520de280ef002847f6a0,https://oven-bucket.pizza4ps.io/images/2023042...,PMH,PMH-02,NaN,2023-04-27 10:44:29.302,2023-04-27 10:44:29.403,0,NaN,NaN,0,...,0,0,0,0,0,0,0,0,0,0
644a5224e280ef002847f6a1,https://oven-bucket.pizza4ps.io/images/2023042...,SGP,SGP-02,NaN,2023-04-27 10:44:52.392,2023-04-27 10:44:52.468,0,NaN,NaN,0,...,0,0,0,0,0,0,0,0,0,0
644a5227e280ef002847f6a2,https://oven-bucket.pizza4ps.io/images/2023042...,XTH,XTH-01,NaN,2023-04-27 10:44:55.313,2023-04-27 10:44:55.388,0,NaN,NaN,0,...,0,0,0,0,0,0,0,0,0,0
644a5244e280ef002847f6a3,https://oven-bucket.pizza4ps.io/images/2023042...,HUB2,HUB2-01,NaN,2023-04-27 10:45:24.402,2023-04-27 10:45:24.462,0,NaN,NaN,0,...,0,0,0,0,0,0,0,0,0,0
644a5249e280ef002847f6a4,https://oven-bucket.pizza4ps.io/images/2023042...,LTE,LTE-01,NaN,2023-04-27 10:45:29.035,2023-04-27 10:45:29.098,0,NaN,NaN,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
all_error_list = ['Bánh không tròn | Distorted shape',
 'Cháy | Baking - Burnt',
 'Viền k đều | Edge - Uneven',
 'Bánh bé | Size - Too small',
 'Thiếu bóng | too few balls',
 'Không đốm | Baking - Does not have leopard-spotting',
 'Màu nhạt | Baking - Pale',
 'Viền nhỏ | Edge - Too small',
 'Viền to | Edge - Too big',
 'Nở viền không đủ | edge pizza is not enough swollen',
 'Viền thấp | Edge - Too low',
 'Phô mai cao | Topping - Cheese too high * with a core*',
 'Topping - Không đúng | incorrect portioning',
 'Không cân Topping - Not even half and half',
 'Sốt trên mép | Topping - Sauce covering on the edge',
 'Lên men thiếu | Fermentation - Lack fermentation',
 'Quá theo viền | Topping - Too strong shaping the edge',
 'Trộn lẫn | Topping - Bended',
 'Quá tập trung | Topping - Topping too centered',
 'Topping - Không đều | Not even',
 'Lên men quá nhiều | Fermentation - Over fermentation',
 'Không hình tròn | Topping - Not circled',
 'Bánh lớn | Size - Too big']

In [8]:
queue_download_image = []
for error in all_error_list:
    error_df = df[df[error] == 1].head(500)
    for image_id, image_url in zip(error_df.index, error_df['image_url']):
        queue_download_image.append((image_id, image_url))

In [5]:
# data_path = 'data/pizza_image'
# downloaded = []
# for id, url in tqdm(queue_download_image):
#     if id not in downloaded:
#         downloaded.append(id)
#         response = requests.get(url)
#         image_data = BytesIO(response.content)
#         image = Image.open(image_data)
#         image_file_name = os.path.join(data_path, f"{id}.jpg")
#         image.save(image_file_name)
#         if response.status_code == 200:
#             pass
#         else:
#             print(f"Failed to download {image_url}")

In [12]:
for id, _ in queue_download_image:
    

1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
